In [3]:
#Descompactando base de dados

!wget "https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip" && unzip semana-1.zip -d dados/

--2022-11-27 12:52:13--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 54.231.202.89, 52.216.138.83, 52.217.96.252, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|54.231.202.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18975214 (18M) [application/zip]
Saving to: ‘semana-1.zip’

semana-1.zip        100%[===================>]  18.10M  61.8MB/s    in 0.3s    

2022-11-27 12:52:14 (61.8 MB/s) - ‘semana-1.zip’ saved [18975214/18975214]

Archive:  semana-1.zip
  inflating: dados/dataset_bruto.json  


In [4]:
!pip install pyspark==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 31 kB/s 
     |████████████████████████████████| 199 kB 45.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=e1cf92614c6e4ea386f74a9c574ea35ba81c697233ea3335d40d22dfe9384d99
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder\
    .master('local[*]') \
    .appName("Transformação com Spark")\
    .getOrCreate()
spark

In [10]:
dados = spark.read.json('/content/dados/dataset_bruto.json')

# Nova seção

In [11]:
dados.show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------+
|anuncio                                                               

Contagem do número de dados

In [12]:
dados.count()

89083

In [13]:
dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [16]:
dados\
  .select(
      'anuncio.*'
  ).show()

DataFrame[andar: bigint, area_total: array<string>, area_util: array<string>, banheiros: array<bigint>, caracteristicas: array<string>, endereco: struct<bairro:string,cep:string,cidade:string,estado:string,latitude:double,longitude:double,pais:string,rua:string,zona:string>, id: string, quartos: array<bigint>, suites: array<bigint>, tipo_anuncio: string, tipo_unidade: string, tipo_uso: string, vaga: array<bigint>, valores: array<struct<condominio:string,iptu:string,tipo:string,valor:string>>]

In [15]:
len(dados.columns)

3

In [30]:
anuncio = dados\
  .select(
      'anuncio.*'
  )

In [18]:
anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

Leitura de imóveis Comercial/Residencial

In [19]:
anuncio\
  .select('tipo_uso')\
  .groupBy('tipo_uso')\
  .count()\
  .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+



In [31]:
anuncio.filter("tipo_uso == 'Residencial'")

DataFrame[andar: bigint, area_total: array<string>, area_util: array<string>, banheiros: array<bigint>, caracteristicas: array<string>, endereco: struct<bairro:string,cep:string,cidade:string,estado:string,latitude:double,longitude:double,pais:string,rua:string,zona:string>, id: string, quartos: array<bigint>, suites: array<bigint>, tipo_anuncio: string, tipo_unidade: string, tipo_uso: string, vaga: array<bigint>, valores: array<struct<condominio:string,iptu:string,tipo:string,valor:string>>]

In [32]:
anuncio\
  .groupBy('tipo_unidade')\
  .count()\
  .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros|11963|
| Apartamento|66801|
|        Casa|10319|
+------------+-----+



In [33]:
anuncio = anuncio.filter("tipo_unidade == 'Apartamento'")

In [34]:
anuncio.filter("tipo_anuncio == 'Usado'").show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{245, null, Vend...|
|    2|      [42]|     [42]|      [1]|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{0, 0, Venda, 15...|
|    1|      [41]|     [41]|      [1]|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

In [35]:
import pyspark.sql.functions as f

In [36]:
anuncio\
  .select(f.size(f.col('quartos')).alias('quartos'))\
  .groupBy('quartos')\
  .count()\
  .show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66651|
|      2|  150|
+-------+-----+



Ctrl * Shift + L (seleciona todas as colunas)

In [39]:
caracteristicas = ['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']

In [40]:
for col in caracteristicas:
  anuncio\
  .select(f.size(f.col(col)).alias(col))\
  .groupBy(col)\
  .count()\
  .show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66651|
|      2|  150|
+-------+-----+

+------+-----+
|suites|count|
+------+-----+
|     1|61154|
|     2|   92|
|     0| 5555|
+------+-----+

+---------+-----+
|banheiros|count|
+---------+-----+
|        1|66700|
|        2|  101|
+---------+-----+

+----+-----+
|vaga|count|
+----+-----+
|   1|63693|
|   2|   90|
|   0| 3018|
+----+-----+

+----------+-----+
|area_total|count|
+----------+-----+
|         1|57383|
|         0| 9418|
+----------+-----+

+---------+-----+
|area_util|count|
+---------+-----+
|        1|66593|
|        2|  208|
+---------+-----+



In [41]:
anuncio\
  .select([f.col(c)[0].alias(c) if c in caracteristicas else c for c in anuncio.columns])\
  .show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|
|    1|        41|       41|        1|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4